In [ ]:
df = pd.read_csv('orders_2.csv')

df.head(5)

<h1> EDA and data filtering

In [ ]:
# Step 1: Handle missing values
# We will fill missing numerical values with the median and categorical values with the mode.
numerical_cols = ['order_item_id', 'price', 'freight_value', 'payment_value']
categorical_cols = ['product_id', 'shipping_limit_date', 'product_category_name_english']

for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Check if there are any missing values left
missing_values = df.isnull().sum()
missing_values

In [ ]:
# Step 2: Visualize the distribution of numerical columns
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 10))
for i, col in enumerate(numerical_cols, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# Step 3: Analyze the categorical columns
plt.figure(figsize=(15, 15))
for i, col in enumerate(categorical_cols, 1):
    plt.subplot(3, 1, i)
    sns.countplot(y=df[col], order=df[col].value_counts().index[:10])
    plt.title(f'Top 10 Product Categories')
    plt.xlabel('Count')
    plt.ylabel(col)
plt.tight_layout()
plt.show()

In [ ]:
# Step 4: Check for any correlations between columns
correlation_matrix = df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Numerical Columns')
plt.show()

In [ ]:
# Step 5: Analyze trends based on the 'order_purchase_timestamp' column
# Convert 'order_purchase_timestamp' to datetime format
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Extract month and year from the timestamp
df['purchase_month'] = df['order_purchase_timestamp'].dt.month
df['purchase_year'] = df['order_purchase_timestamp'].dt.year

# Group by month and year to get the number of orders per month
monthly_orders = df.groupby(['purchase_year', 'purchase_month']).size().reset_index(name='number_of_orders')

# Plotting the trend
plt.figure(figsize=(15, 7))
sns.lineplot(x='purchase_month', y='number_of_orders', hue='purchase_year', data=monthly_orders, marker='o')
plt.title('Number of Orders per Month')
plt.xlabel('Month')
plt.ylabel('Number of Orders')
plt.legend(title='Year')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
# First, let's identify repeat purchasers
repeat_purchasers = df['customer_unique_id'].value_counts()[df['customer_unique_id'].value_counts() > 1].index

# Filter the dataset for repeat purchasers
repeat_purchasers_df = df[df['customer_unique_id'].isin(repeat_purchasers)]

# Group by product category and count the number of repeat purchasers for each category
category_repeat_purchase = repeat_purchasers_df.groupby('product_category_name_english')['customer_unique_id'].nunique().reset_index()
category_repeat_purchase = category_repeat_purchase.sort_values(by='customer_unique_id', ascending=False)

# Plotting the top 10 product categories with the highest number of repeat purchasers
plt.figure(figsize=(12, 8))
sns.barplot(y='product_category_name_english', x='customer_unique_id', data=category_repeat_purchase.head(10), palette='viridis')
plt.title('Top 10 Product Categories with Highest Number of Repeat Purchasers')
plt.xlabel('Number of Repeat Purchasers')
plt.ylabel('Product Category')
plt.show()

In [ ]:
# Customer Lifetime Value (LTV) Analysis by Product

# Calculate the total payment value for each customer and product combination
customer_product_value = df.groupby(['customer_unique_id', 'product_category_name_english'])['payment_value'].sum().reset_index()

# Calculate the average LTV for each product category
product_ltv = customer_product_value.groupby('product_category_name_english')['payment_value'].mean().reset_index()
product_ltv = product_ltv.sort_values(by='payment_value', ascending=False)

# Plotting the LTV for each product category
plt.figure(figsize=(12, 10))
sns.barplot(y='product_category_name_english', x='payment_value', data=product_ltv.head(10), palette='magma')
plt.title('Top 10 Product Categories by Customer LTV')
plt.xlabel('Average Customer LTV')
plt.ylabel('Product Category')
plt.show()

<h1> Customer Analysis Begins

In [ ]:
# Customer Analysis

# 1. LTV and Repeat Purchase Rate
# Calculate LTV for each customer
customer_ltv = df.groupby('customer_unique_id')['payment_value'].sum().reset_index()
average_ltv = customer_ltv['payment_value'].mean()

# Calculate Repeat Purchase Rate
total_customers = df['customer_unique_id'].nunique()
repeat_customers = df['customer_unique_id'].value_counts()[df['customer_unique_id'].value_counts() > 1].nunique()
repeat_purchase_rate = repeat_customers / total_customers

average_ltv, repeat_purchase_rate

#make CSV

(213.02371232933734, 0.000280969030969031)

In [ ]:
# Analyzing LTV and repeat purchase rate based on the first purchase
# Identify the first purchase for each customer
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
first_purchase = df.groupby('customer_unique_id').agg({'order_purchase_timestamp': 'min'})
first_purchase_df = df.merge(first_purchase, on=['customer_unique_id', 'order_purchase_timestamp'])

# Calculate LTV for each product category based on the first purchase
first_purchase_ltv = first_purchase_df.groupby('product_category_name_english')['payment_value'].mean().reset_index()
first_purchase_ltv = first_purchase_ltv.sort_values(by='payment_value', ascending=False)

# Calculate Repeat Purchase Rate for each product category based on the first purchase
repeat_purchase_by_product = first_purchase_df.groupby('product_category_name_english')['customer_unique_id'].count().reset_index()
repeat_purchase_by_product = repeat_purchase_by_product.sort_values(by='customer_unique_id', ascending=False)

first_purchase_ltv.head(10)

In [ ]:
repeat_purchase_by_product.head(10)

In [ ]:
!pip install -q mlxtend

To do:
1. Do repeat purchasers have a higher LTV than single time customers
2. Next order prediction
3. Churn modelling
4. Predicted LTV

In [ ]:
# Group by customer_unique_id and count the number of orders using the dataframe named 'DF'
customer_orders = df.groupby('customer_unique_id').agg(num_orders=('order_id', 'nunique'))

# Identify repeat and one-time purchasers
customer_orders['purchaser_type'] = 'one-time'
customer_orders.loc[customer_orders['num_orders'] > 1, 'purchaser_type'] = 'repeat'

# Display the first few rows of the dataframe
customer_orders

In [ ]:
# Group by customer_unique_id and count the number of orders using the dataframe named 'df'
customer_orders = df.groupby('customer_unique_id').agg(num_orders=('order_id', 'nunique'))

# Identify repeat and one-time purchasers
customer_orders['purchaser_type'] = 'one-time'
customer_orders.loc[customer_orders['num_orders'] > 1, 'purchaser_type'] = 'repeat'

# Display the first few rows of the dataframe
customer_orders.head()

In [ ]:
# Merge the main dataframe with the customer_orders dataframe to get the purchaser_type for each customer
merged_df = df.merge(customer_orders[['purchaser_type']], left_on='customer_unique_id', right_index=True)

# Calculate the LTV for each group
ltv_data = merged_df.groupby(['customer_unique_id', 'purchaser_type']).agg(total_purchase_value=('payment_value', 'sum')).reset_index()
ltv_avg = ltv_data.groupby('purchaser_type').agg(avg_LTV=('total_purchase_value', 'mean'))

# Display the average LTV for each group
ltv_avg

#make csv

In [ ]:
# Convert the 'order_purchase_timestamp' to a datetime format
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Extract the month and year from the 'order_purchase_timestamp' to create a 'month_year' column
df['month_year'] = df['order_purchase_timestamp'].dt.to_period('M')

# Merge the main dataframe with the customer_orders dataframe to get the purchaser_type for each customer
merged_df = df.merge(customer_orders[['purchaser_type']], left_on='customer_unique_id', right_index=True)

# Calculate the LTV for each group and time period
ltv_data = merged_df.groupby(['month_year', 'customer_unique_id', 'purchaser_type']).agg(total_purchase_value=('price', 'sum')).reset_index()
ltv_avg = ltv_data.groupby(['month_year', 'purchaser_type']).agg(avg_LTV=('total_purchase_value', 'mean'))

# Display the average LTV for each group and time period
ltv_avg


In [ ]:
# Convert DataFrame to CSV
ltv_avg.to_csv('ltv_time.csv', index=False)

In [ ]:
# Filter the merged dataframe to get only repeat purchasers
repeat_purchasers = merged_df[merged_df['purchaser_type'] == 'repeat']

# Sort the dataframe by customer_unique_id and order_purchase_timestamp to get the first order for each customer
repeat_purchasers_sorted = repeat_purchasers.sort_values(by=['customer_unique_id', 'order_purchase_timestamp'])

# Drop duplicate customer_unique_id to keep only the first order for each customer
first_orders = repeat_purchasers_sorted.drop_duplicates(subset='customer_unique_id', keep='first')

# Group by product_category_name_english to get the count of products bought on the first order
products_first_order = first_orders.groupby('product_category_name_english').size().reset_index(name='count')
products_first_order_sorted = products_first_order.sort_values(by='count', ascending=False)

# Display the products bought by repeat purchasers on their first order
products_first_order_sorted

In [ ]:
# Filter out customers who only made one purchase
repeat_customers = sorted_df['customer_unique_id'].value_counts()[sorted_df['customer_unique_id'].value_counts() > 1].index
repeat_customers_df = sorted_df[sorted_df['customer_unique_id'].isin(repeat_customers)]

# Get the first and second orders for these repeat customers
first_orders = repeat_customers_df.groupby('customer_unique_id').nth(0).reset_index()
second_orders = repeat_customers_df.groupby('customer_unique_id').nth(1).reset_index()

# Merge the first and second orders dataframes
merged_orders = first_orders.merge(second_orders[['customer_unique_id', 'product_category_name_english']], on='customer_unique_id', suffixes=('_first', '_second'))

# Display the merged dataframe
merged_orders.head()

In [ ]:
# Check for missing values in the product categories
missing_first_order_categories = merged_orders['product_category_name_english_first'].isnull().sum()
missing_second_order_categories = merged_orders['product_category_name_english_second'].isnull().sum()

missing_first_order_categories, missing_second_order_categories

(0, 0)

In [ ]:
# Group by product_category_name_english and product_id to get the count of each product sold
product_popularity = df.groupby(['product_category_name_english', 'product_id']).size().reset_index(name='count')

# Sort the products in each category by their popularity
product_popularity_sorted = product_popularity.sort_values(by=['product_category_name_english', 'count'], ascending=[True, False])

# For each category, get the top 5 most popular products
top_products_per_category = product_popularity_sorted.groupby('product_category_name_english').head(5)

# Display the top products for each category
top_products_per_category

In [ ]:
# Select a random customer's first order
random_customer_first_order = df.sort_values(by='order_purchase_timestamp').drop_duplicates(subset='customer_unique_id', keep='first').sample(1)
customer_id = random_customer_first_order['customer_unique_id'].values[0]

# Get the product categories from the customer's first order
purchased_categories = random_customer_first_order['product_category_name_english'].tolist()

# Recommend the top products from those categories
recommended_products = top_products_per_category[top_products_per_category['product_category_name_english'].isin(purchased_categories)]

# Display the recommended products for the customer
recommended_products

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Data Preparation
# Create a column to indicate repeat purchaser
df['is_repeat'] = df['customer_unique_id'].isin(customer_orders[customer_orders['purchaser_type'] == 'repeat'].index).astype(int)

# Feature Engineering
# Number of unique categories purchased by the customer
unique_categories = df.groupby('customer_unique_id')['product_category_name_english'].nunique().reset_index()
unique_categories.columns = ['customer_unique_id', 'num_unique_categories']

# Total number of purchases by the customer
total_purchases = df.groupby('customer_unique_id').size().reset_index(name='total_purchases')

# Merge the features into a single dataframe
features = df[['customer_unique_id', 'is_repeat']].drop_duplicates()
features = features.merge(unique_categories, on='customer_unique_id', how='left')
features = features.merge(total_purchases, on='customer_unique_id', how='left')

# Split the data into training and test sets
X = features[['num_unique_categories', 'total_purchases']]
y = features['is_repeat']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Building
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep

(0.9801248699271592,
 '              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99     18593\n           1       0.75      0.58      0.66       627\n\n    accuracy                           0.98     19220\n   macro avg       0.87      0.79      0.82     19220\nweighted avg       0.98      0.98      0.98     19220\n')

In [ ]:
# Extract feature importances from the model
feature_importances = pd.DataFrame(clf.feature_importances_, index=X_train.columns, columns=['importance']).sort_values('importance', ascending=False)

# Get the top 10 product categories leading to repeat purchasing
top_categories = df[df['is_repeat'] == 1]['product_category_name_english'].value_counts().head(10).index.tolist()

# For each of the top categories, get the most popular products as the predicted next order
predicted_next_order = top_products_per_category[top_products_per_category['product_category_name_english'].isin(top_categories)].groupby('product_category_name_english').head(1)

predicted_next_order

<h1> Churn Prediction

In [ ]:
# Convert order_purchase_timestamp to datetime format
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Calculate the maximum purchase date for each customer
last_purchase_date = df.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
last_purchase_date.columns = ['customer_unique_id', 'last_purchase_date']

# Calculate the difference between the last purchase date and the maximum date in the dataset
last_purchase_date['days_since_last_purchase'] = (df['order_purchase_timestamp'].max() - last_purchase_date['last_purchase_date']).dt.days

# Classify customers as churned if they haven't made a purchase in the last 90 days
last_purchase_date['is_churned'] = last_purchase_date['days_since_last_purchase'].apply(lambda x: 1 if x > 90 else 0)

# Display the first few rows of the dataframe
last_purchase_date.head()

In [ ]:
# Merge the churn data with the main dataframe to get product categories associated with each customer
df_churn = df.merge(last_purchase_date[['customer_unique_id', 'is_churned']], on='customer_unique_id', how='left')

# Calculate the churn rate for each product category
product_churn_rate = df_churn.groupby('product_category_name_english')['is_churned'].mean().reset_index()
product_churn_rate.columns = ['product_category_name_english', 'churn_rate']

# Sort the product categories by churn rate
product_churn_rate_sorted = product_churn_rate.sort_values(by='churn_rate', ascending=False)

# Get the products with the highest and lowest churn rates
highest_churn_products = product_churn_rate_sorted.head(5)
lowest_churn_products = product_churn_rate_sorted.tail(5)

highest_churn_products, lowest_churn_products

(   product_category_name_english  churn_rate
 67        tablets_printing_image         1.0
 29     fashion_childrens_clothes         1.0
 11             cds_dvds_musicals         1.0
 61         security_and_services         1.0
 52                    la_cuisine         1.0,
             product_category_name_english  churn_rate
 18              construction_tools_lights    0.709265
 36                                   food    0.699620
 64  small_appliances_home_oven_and_coffee    0.666667
 58                         party_supplies    0.630435
 3                   arts_and_craftmanship    0.250000)

In [ ]:
highest_churn_products

In [ ]:
lowest_churn_products

In [ ]:
# Feature Engineering for Churn Prediction

# Average time between purchases for each customer
df_churn['order_purchase_timestamp'] = pd.to_datetime(df_churn['order_purchase_timestamp'])
df_churn_sorted = df_churn.sort_values(['customer_unique_id', 'order_purchase_timestamp'])
df_churn_sorted['time_since_last_order'] = df_churn_sorted.groupby('customer_unique_id')['order_purchase_timestamp'].diff().dt.days
avg_time_between_purchases = df_churn_sorted.groupby('customer_unique_id')['time_since_last_order'].mean().reset_index()

# Merge the new feature with the existing features
features_churn = features.merge(avg_time_between_purchases, on='customer_unique_id', how='left')

# Fill NaN values (for customers with only one purchase) with a large value
features_churn['time_since_last_order'].fillna(999, inplace=True)

# Split the data into training and test sets
X_churn = features_churn[['num_unique_categories', 'total_purchases', 'time_since_last_order']]
y_churn = features_churn['is_repeat']
X_train_churn, X_test_churn, y_train_churn, y_test_churn = train_test_split(X_churn, y_churn, test_size=0.2, random_state=42)

# Model Building for Churn Prediction
clf_churn = RandomForestClassifier(random_state=42)
clf_churn.fit(X_train_churn, y_train_churn)
y_pred_churn = clf_churn.predict(X_test_churn)

# Model Evaluation
accuracy_churn = accuracy_score(y_test_churn, y_pred_churn)
classification_rep_churn = classification_report(y_test_churn, y_pred_churn)

accuracy_churn, classification_rep_churn

(0.9906867845993756,
 '              precision    recall  f1-score   support\n\n           0       0.99      1.00      1.00     18593\n           1       1.00      0.72      0.83       627\n\n    accuracy                           0.99     19220\n   macro avg       0.99      0.86      0.91     19220\nweighted avg       0.99      0.99      0.99     19220\n')

In [ ]:
# Extract feature importances from the churn model
feature_importances_churn = pd.DataFrame(clf_churn.feature_importances_, index=X_train_churn.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances_churn

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters and their possible values
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV with the model and hyperparameters
grid_search = GridSearchCV(estimator=clf_churn, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the model to the training data
grid_search.fit(X_train_churn, y_train_churn)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
best_params

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimato

[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_esti

[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=10, n_estima

{'max_depth': None,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 50}

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Train the model with the best hyperparameters
optimized_clf_churn = RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)
optimized_clf_churn.fit(X_train_churn, y_train_churn)

# Predict on the test set
y_pred_churn = optimized_clf_churn.predict(X_test_churn)

# Evaluate the model
accuracy = accuracy_score(y_test_churn, y_pred_churn)
classification_rep = classification_report(y_test_churn, y_pred_churn)

accuracy, classification_rep

(0.9906867845993756,
 '              precision    recall  f1-score   support\n\n           0       0.99      1.00      1.00     18593\n           1       1.00      0.72      0.83       627\n\n    accuracy                           0.99     19220\n   macro avg       0.99      0.86      0.91     19220\nweighted avg       0.99      0.99      0.99     19220\n')

<h1> RFM ANALYSIS

In [ ]:
# Load the CSV file
df = pd.read_csv('orders_2.csv')

# Convert 'order_purchase_timestamp' column to datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Calculate Recency, Frequency, and Monetary values for each customer

# Recency
recency = df.groupby('customer_unique_id')['order_purchase_timestamp'].max().reset_index()
recency.columns = ['customer_unique_id', 'last_purchase_date']
recency['recency'] = (df['order_purchase_timestamp'].max() - recency['last_purchase_date']).dt.days

# Frequency
frequency = df.groupby('customer_unique_id')['order_id'].count().reset_index()
frequency.columns = ['customer_unique_id', 'frequency']

# Monetary
monetary = df.groupby('customer_unique_id')['payment_value'].sum().reset_index()
monetary.columns = ['customer_unique_id', 'monetary']

# Merge the dataframes
rfm = recency.merge(frequency, on='customer_unique_id').merge(monetary, on='customer_unique_id')
rfm = rfm[['customer_unique_id', 'recency', 'frequency', 'monetary']]

rfm.head()

In [ ]:
# Correcting the Monetary calculation using the 'price' column

# Monetary
monetary = df.groupby('customer_unique_id')['price'].sum().reset_index()
monetary.columns = ['customer_unique_id', 'monetary']

# Merge the dataframes
rfm = recency.merge(frequency, on='customer_unique_id').merge(monetary, on='customer_unique_id')
rfm = rfm[['customer_unique_id', 'recency', 'frequency', 'monetary']]

rfm.head()